In [5]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

params = {"ot": "OTTESTING01",
            "datadate": "2023-05-11"}
item_activity_file = "data/input/Matriz de Conversion Clarificador Rev.B1.xlsx"

In [2]:
from settings import settings
from utils.add_columns_to_dataframe import AddColumnsToDataFrame

from excel_processor.excel_processor import ExcelProcessor
from excel_processor.schemas import itemizado_schema, activity_schema

from utils.dataframe_comparator import DataFrameComparator, ComparisonType, ComparisonSchema

from utils.dataframe_aggregator import GroupingParameters, Aggregation, group_dataframe


---
Creación de dataframes de Itemizado y Actividades

In [3]:
# Crea datadrame desde hoja itemizado
processor_item = ExcelProcessor(item_activity_file, itemizado_schema)
item_result = processor_item.process()

if item_result.error_message:
    print(f"Error: {item_result.error_message}")
else:
    item_summary = item_result.dataframe

In [4]:
# Crea dataframe desde la hoja actividad
processor = ExcelProcessor(item_activity_file, activity_schema)
result = processor.process()

if result.error_message:
    print(f"Error: {result.error_message}")
else:
    activity_summary = result.dataframe

---
Validaciones entre dataframes

In [ ]:
# Validar que los items definidos en actividades estén en itemizado
comparator_item = DataFrameComparator( # Comparador df1 a df2
    df1=activity_summary,
    df1_name="activity_summary",
    df2=item_summary,
    df2_name="item_summary"
)

comparison_item_schema = ComparisonSchema(
    comparison_type=ComparisonType.INCLUSION,
    columns_dict={"item_id": "item_id"}  # Mapa de columnas df1 a df2
)

column_validation_result = comparator_item.validate_columns(comparison_item_schema)

if column_validation_result.has_error:
    print(f"Error: {column_validation_result.error_message}")
else:
    value_validation_result = comparator_item.validate_values(comparison_item_schema)
    if value_validation_result.has_error:
        print(f"Error: {value_validation_result.error_message}")


___
Dataframe con las HH Bac de la actividad en base a la matriz de itemizado


In [7]:
# Crear una instancia de la clase GroupingParameters
grouping_params_activity = GroupingParameters(
    df=activity_summary, # DataFrame de origen
    groupby_columns=['activity_code', 'activity_unit'], # Columnas por las cuales agrupar
    aggregations=[ # Agregaciones a realizar:
        Aggregation(column='activity_laborunits', agg_func='sum', output_name='bac_activity'),
        Aggregation(column='qty_activity', agg_func='sum', output_name='qty_activity')
    ]
)
# Llamar a la función group_dataframe con los parámetros de agrupación que acabas de definir
grouping_activity = group_dataframe(grouping_params_activity)

if grouping_activity.error_message:
    print(f"Error: {grouping_activity.error_message}")
else:
    grouped_activity = grouping_activity.dataframe


---
Agrega parametros como columnas a dataframe

In [5]:
# Crear instancia de la clase e ingresar el dataframe y el diccionario de parámetros
add_columns_to_df = AddColumnsToDataFrame(dataframe=item_summary, parameters=params) # type: ignore

# Agregar columnas al dataframe
item_summary_with_columns = add_columns_to_df.add_columns()

___
Creación de dataframe desde sql server (P6)

In [9]:
from p6_data.sql_data_frame_creator import SQLDataFrameCreator, SQLParams

activitys_sql = "sql_querys/activitys_p6.sql"

# Define los parámetros SQL
params = SQLParams(params=settings.SQL_PARAMS)

# Crea el creador de dataframe
creator = SQLDataFrameCreator(query_file=activitys_sql, params=params)

# Crea el dataframe
activitys_p6 = creator.get_dataframe()


2023-05-22 20:40:37 [INFO]: Running SQL query...
/Users/miguelangelgonzalezalfaro/opt/anaconda3/envs/py3_10/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
2023-05-22 20:40:37 [INFO]: SQL query successfully run.


In [10]:
activitys_p6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID Actividad              82 non-null     object 
 1   total_hh_actividad        82 non-null     float64
 2   activity_percentcomplete  82 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.0+ KB
